<a href="https://colab.research.google.com/github/nneibaue/alumni_shuffler/blob/nate_testing/Alumni_Shuffler_Official.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Alumni Shuffler</h1>


Hello all! Welcome to Google Colab, this awesome tool for sharing Python notebooks in Google Drive. This has been a fun project to work on, and we hope that it is a useful! We live in a strange time, and are lucky to have technology that allows to stay connected while apart. Not only is it a tool, but this project is an example of two people in different states who have never met in person collaborating on a project together -- a project whose sole purpose is to help facilitate the virtual interaction of people from all over the country.  

<br>

---

<br>

The Alumni Shuffler is a tool that helps intelligently create breakout groups during large Zoom calls. The project is in its infancy right now, and is currently being developed by Nathan Neibauer and Hayden Blair. 

Given a virtual Zoom event with upwards of 20 or 30 people, identified by certain characterics ('track', 'year', 'hair color', 'likes _The Office_ ', etc.), this tool can help ensure that everyone can...

* Interact with everyone else in their group
* Interact with as many different people as possible

...without eating up too much mental real-estate from the coordinator, who likely wants to spend more time interacting with students and less time fussing with a spreadsheet. 

Alumni data must be saved in a spreadsheet somewhere on Google Drive for the user who is operating this notebook with columns representing the categories to sort by. An example might look like this

<br>

Name | track | year | hair_color | hard_working |
---|---|---| --- | ---
Leslie | optics | 2017 | blonde | yes
Ron | polymer | 2014 | brown | no
Jean Ralphio | semi | 2012 | black | no
April | sensors | 2019 | black | yes
Ann | semi | 2013 | brown | yes

<br>

<font color=red>Files must be .xlsx. We are working to support google sheets, but it is not ready yet! If your file is a google sheets file, you must download it as .xlsx and then re-upload (sorry!)</font>

For now, I would imagine that 'track' and 'year' are the only identifiers, but the code should work with any number of them.

### Imports

The following cell clones the github repo so private libraries can be imported.

In [ ]:
#@title Clone Github Repo

BRANCH_NAME = "nate_testing" #@param {type:"string"}

import os
import sys
import shutil

ROOT = '/content'
os.chdir(ROOT)
REPO_NAME = 'alumni_shuffler'
REPO_URL = f'https://github.com/nneibaue/{REPO_NAME}'
REPO_PATH = os.path.join(ROOT, REPO_NAME)


# Remove old repo
print('Removing old repo...')
!rm -rf $REPO_PATH

print('Cloning from github...')
!git clone $REPO_URL
os.chdir(REPO_PATH)

if BRANCH_NAME != 'master':
  !git checkout --track origin/$BRANCH_NAME
  !git config user.email "colab_anon@gmail.com"
else:
  !git pull
  
if REPO_PATH not in sys.path:
  print(f'Adding {REPO_PATH} to path')
  sys.path.append(REPO_PATH)

os.chdir(ROOT)

In [ ]:
#@title Other imports / Mount Google Drive
from html_maker import HtmlMaker
import zoom_sesh
import time
import ipywidgets as iw
from google.colab import drive, widgets, output
from IPython.display import display, HTML

COLAB_ROOT='/content'

# Google drive mounting
drive.mount('/content/gdrive')
DRIVE_BASE=('/content/gdrive/My Drive')


In [ ]:
#@title Zoom Session Manager


#@markdown Enter path to a directory containing 'alumni.xlsx' file
session_directory = "" #@param {type:"string"}
dev_mode = False #@param {type:"boolean"}
#@markdown Dev mode Settings:
max_people = 40 #@param {type:"integer"}

def make_breakout_ui(z):
  breakout_output = iw.Output()
  #status_indicator = iw.Output(layout=iw.Layout(width='300px', padding='5px', border='1px solid black'))
  status_indicator = iw.Output(layout=iw.Layout(padding='20px'))
  with status_indicator:
    display(iw.HTML('<font size="20" color="#085337">Ready</font>'))

  num_breakouts = 0
  smalltextbox = iw.Layout(width='200px', height='25px')
  breakout_button = iw.Button(description='Breakout',
                              layout=iw.Layout(border='5x solid green'),
                              style={'button_color':"#007030"})
  breakout_button.add_class('green-button')

  by_selector = iw.Dropdown(options=['track', 'year', 'any'],
                            layout=iw.Layout(width='max-content'),
                            description='Group by')

  group_size_selector = iw.Textarea(description='Group size', value='5', layout=smalltextbox)
  diff_selector = iw.ToggleButtons(options=['Same', 'Difference'])
  controls = iw.HBox([breakout_button,
                      by_selector,
                      group_size_selector,
                      diff_selector,
                      status_indicator],
                     layout=iw.Layout(
                         width='100%',
                         height='70px',))
                         #border='2px solid black'))
#   {'extras': [],
#  'group1': (5, 6, 8, 18),
#  'group2': (0, 7, 17, 19),
#  'group3': (2, 4, 10, 11),
#  'group4': (1, 9, 12, 13),
#  'group5': (3, 14, 15, 16)} 

  def update_from_excel(b):
    i = len(z._breakout_history) - 1
    latest_breakout = z._breakout_history[-1]


  def show_breakout_groups(breakout):
    inner_maker = HtmlMaker()
    cols = ['name', 'year', 'track']
      
    inner_maker.add_html_element(f'<h2>Breakout {num_breakouts}</h2>')
    for group in breakout:
      inner_maker.add_pandas_df(z.alumni.iloc[breakout[group]][cols],
                                td_class="",
                                title=group,
                                enclosing_tag='div',
                                css_classes=["horizontal-table"])

    #maker.apply_style({'table': {'display': 'inline-table'}})
    element = iw.HTML(inner_maker.to_html())
    with breakout_output:
      display(element)
    status_indicator.clear_output()
    with status_indicator:
      display(iw.HTML('<font size="20" color="#085337">Ready</font>'))


  def perform_breakout(b):
    nonlocal num_breakouts
    num_breakouts += 1
    status_indicator.clear_output()
    with status_indicator:
      display(iw.HTML('<font size="22" color="#fee11a">Working....</font>'))
    # `by_selector` is an HBox, so this grabs the actual dropdown value
    by = by_selector.value

    if by == 'any':
      by = 'all'

    group_size = int(group_size_selector.value)
    if diff_selector.value == 'Same':
      diff = False
    else:
      diff = True

    breakout = z.breakout(by=by, group_size=group_size, diff=diff)
    show_breakout_groups(breakout)

    

  # Convert to ipywidgets HTML element
  summary_html = iw.HTML(z.summary_html())

  style = iw.HTML('<style>.green-button {color: white}</style>')

  app = iw.VBox([summary_html,
                 breakout_output,
                 controls,
                 style],)

                #layout=iw.Layout(padding='10px'))
  
  breakout_button.on_click(perform_breakout)
  
  display(app)

if dev_mode:
  zoom_sesh.make_fake_data(session_directory)
  time.sleep(5)
else:
  session_directory = os.path.join(DRIVE_BASE, session_directory)
z = zoom_sesh.ZoomSesh(session_directory)
make_breakout_ui(z)